In [48]:
!pip install cdsapi numpy urllib3  pandas datetime xarray[complete] 

Defaulting to user installation because normal site-packages is not writeable
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)


In [1]:
import xarray as xr
# plt.style.available
#import colormaps as cmaps 
#import concurrent
import numpy as np
# # Libraries to assist with animation and visualisations
# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib import animation
from IPython.display import HTML

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()

import json
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
# import cfgrib
# import eccodes
# import geopandas as gpd
# from shapely.geometry import Point
import zipfile
import os

from zipfile import ZipFile 


In [2]:
from datetime import date,timedelta
time1=(date.today()- timedelta(days = 1)).strftime("%Y-%m-%d")
time1


'2025-01-02'

In [ ]:
import cdsapi
URL = 'https://ads-beta.atmosphere.copernicus.eu/api'
KEY = '87bb8d36-2458-48c2-b5e3-f1fd352291bc'
dataset = "cams-global-atmospheric-composition-forecasts"
lead= [str(i) for i in range(0,73)]

request = {
    "date": [f"{time1}/{time1}"],
    "time": ["12:00"],
    "leadtime_hour": lead,
    "type": ["forecast"],
    "data_format": "netcdf_zip",
    "variable": [
         "2m_dewpoint_temperature",
        "2m_temperature",
        "dust_aerosol_optical_depth_550nm",
        "mean_sea_level_pressure",
        "nitrate_aerosol_optical_depth_550nm",
        "particulate_matter_2.5um",
        "particulate_matter_10um",
        "sea_salt_aerosol_optical_depth_550nm",
        "sulphate_aerosol_optical_depth_550nm",
        "total_aerosol_optical_depth_550nm",
        "total_column_water_vapour",
        "convective_precipitation",
        "high_cloud_cover",
        "low_cloud_cover",
        "medium_cloud_cover",
        "total_cloud_cover",
        "total_precipitation",
        "most_unstable_convective_available_potential_energy",
        "most_unstable_convective_inhibition"

    ],
    "area": [50, 50, 0, 100]
}

client = cdsapi.Client(url=URL, key=KEY)
client.retrieve(dataset, request).download('test.zip')


In [5]:

with ZipFile("test.zip", 'r') as zObject: 
  
    # Extracting specific file in the zip 
    # into a specific location. 
    zObject.extractall( 
         path=".") 
zObject.close() 

ds=xr.open_dataset('data_sfc.nc')

In [8]:
ds.valid_time.values

array([['2025-01-02T12:00:00.000000000', '2025-01-02T13:00:00.000000000',
        '2025-01-02T14:00:00.000000000', '2025-01-02T15:00:00.000000000',
        '2025-01-02T16:00:00.000000000', '2025-01-02T17:00:00.000000000',
        '2025-01-02T18:00:00.000000000', '2025-01-02T19:00:00.000000000',
        '2025-01-02T20:00:00.000000000', '2025-01-02T21:00:00.000000000',
        '2025-01-02T22:00:00.000000000', '2025-01-02T23:00:00.000000000',
        '2025-01-03T00:00:00.000000000', '2025-01-03T01:00:00.000000000',
        '2025-01-03T02:00:00.000000000', '2025-01-03T03:00:00.000000000',
        '2025-01-03T04:00:00.000000000', '2025-01-03T05:00:00.000000000',
        '2025-01-03T06:00:00.000000000', '2025-01-03T07:00:00.000000000',
        '2025-01-03T08:00:00.000000000', '2025-01-03T09:00:00.000000000',
        '2025-01-03T10:00:00.000000000', '2025-01-03T11:00:00.000000000',
        '2025-01-03T12:00:00.000000000', '2025-01-03T13:00:00.000000000',
        '2025-01-03T14:00:00.000000000

In [15]:
aod_du=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['duaod550']
aod_ss=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['ssaod550']
aod_su=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['suaod550']
aod=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['aod550']
aod_ni=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['niaod550']
time=ds.valid_time






rf1 = ds.sel(longitude=slice(50,100),latitude=slice(40,0))['tp'][:, :, :]  # Select a specific time slice
p =ds.sel(longitude=slice(50,100),latitude=slice(40,0))['msl'][:, :, :]/100  # Select a specific time slice
tcw=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['tcwv']
pm25=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm2p5']
pm10=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['pm10']
t2=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['t2m']

lcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['lcc']
mcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mcc']
hcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['hcc']
tcc=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['tcc']

cape=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucape']
cin=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['mucin']

d2m=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['d2m']
cp=ds.sel(longitude=slice(50,100),latitude=slice(40,0))['cp']


# w10=(u10**2+v10**2)**0.5
# from metpy.units import units
# rf2=rf* units('m').to(units.mm)
rf=rf1.diff(dim='forecast_period').dropna(dim='forecast_period')

In [ ]:
aod_ss

In [16]:

def dict_l(data,fm):
  data_dict=data.squeeze().to_dict(data='list', encoding=False)
  def convert_types(obj):
    if isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
        return obj.isoformat()  # Convert datetime objects to ISO format
    elif isinstance(obj, timedelta):
        return obj.total_seconds()  # Convert timedelta to total seconds
    elif isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert numpy arrays to lists
    elif isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert numpy floats to native floats
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert numpy integers to native ints
    raise TypeError(f"Type {type(obj)} not serializable")

  file=f"{fm}.json"
  # Save the dictionary as a JSON file with the extended convert_types function
  with open(file, 'w') as json_file:
      json.dump(data_dict, json_file, indent=4, default=convert_types)
  print('done',file)

In [17]:
 dict_l(t2-273.15,"2m_temp")
 dict_l(rf*1000,"rf")
 dict_l(p,"msl")
 dict_l(tcw,"tcwv")
 dict_l(1e9*pm25,"PM25")
 dict_l(1e9*pm10,"PM10")

 dict_l(lcc,"LCC")
 dict_l(mcc,"MCC")
 dict_l(hcc,"HCC")
 dict_l(tcc,"TCC")

 dict_l(cape,"CAPE")
 dict_l(cin,"CIN")
 dict_l(d2m-273.15,"D2M")
 dict_l(cp*1000,"CP")

 dict_l(aod_ss,"AOD_SEA")
 dict_l(aod_su,"AOD_SUL")
 dict_l(aod_ni,"AOD_NIT")
 dict_l(aod_du,"AOD_DUST")
 dict_l(aod,"AOD_TOT")

dict_l(time,"TIME")






done TIME.json


In [ ]:
# import numpy as np

# # Define the colors in RGBA format
# colors = {
#     "blue": (0, 0, 255, 1),
#     "aqua": (0, 255, 255, 1),
#     "green": (0, 255, 0, 1),
#     "yellow": (255, 255, 0, 1),
#     "orange": (255, 165, 0, 1),
#     "red": (255, 0, 0, 1)
# }

# # Define color stops and corresponding positions
# color_stops = [
#     (0.0, (0, 0, 0, 1)),  # Changed to tuple
#     (0.12, (0, 0, 255, 1)),  # Changed to tuple
#     (0.25, (0, 255, 255, 1)),  # Changed to tuple
#     (0.38, (0, 255, 0, 1)),  # Changed to tuple
#     (0.5, (255, 255, 0, 1)),  # Changed to tuple
#     (0.62, (255, 165, 0, 1)),  # Changed to tuple
#     (0.75, (255, 0, 0, 1)),  # Changed to tuple
#     (0.88, (238, 130, 238, 1)),  # Changed to tuple
#     (1.0, (255, 255, 255, 1))  # Changed to tuple
# ]

# # Generate 100 interpolated values
# n_points = 200
# interpolated_colors = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest stops
#     for j in range(len(color_stops) - 1):
#         if color_stops[j][0] <= pos <= color_stops[j + 1][0]:
#             # Interpolate between the two stops
#             start_pos, start_color = color_stops[j]
#             end_pos, end_color = color_stops[j + 1]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             interpolated_colors.append([round(pos, 2), rgba_string])
#             break

# interpolated_colors


In [ ]:
# # Number of interpolated colors
# n_points = 256

# # Prepare an empty list for the AMWG256 colormap
# amwg256 = []

# # Define the positions of the input colors in the sequence
# color_positions = [i / (len(color_sequence) - 1) for i in range(len(color_sequence))]

# # Generate interpolated colors for 256 points
# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(color_positions) - 1):
#         if color_positions[j] <= pos <= color_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = color_positions[j], color_positions[j + 1]
#             start_color, end_color = color_map[color_sequence[j]], color_map[color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             amwg256.append([round(pos, 3), rgba_string])
#             break

# amwg256  # Display the first 10 color stops to verify


In [ ]:
# # Define the colormap structure for GMT_panoply with stops: Black, Blue, Aqua, Green, Yellow, Orange, Red, White
# gmt_color_sequence = ["Bk", "Bl", "Aq", "Gr", "Ye", "Or", "Re", "Wh"]

# # Define the positions of the input colors
# gmt_positions = [i / (len(gmt_color_sequence) - 1) for i in range(len(gmt_color_sequence))]

# # Interpolating 256 colors for the GMT_panoply colormap
# gmt_panoply_256 = []

# for i in range(n_points):
#     pos = i / (n_points - 1)
#     # Find the two closest positions in the sequence
#     for j in range(len(gmt_positions) - 1):
#         if gmt_positions[j] <= pos <= gmt_positions[j + 1]:
#             # Interpolate between the two colors
#             start_pos, end_pos = gmt_positions[j], gmt_positions[j + 1]
#             start_color, end_color = color_map[gmt_color_sequence[j]], color_map[gmt_color_sequence[j + 1]]
#             ratio = (pos - start_pos) / (end_pos - start_pos)
#             interpolated_color = tuple(
#                 int(start_color[k] + ratio * (end_color[k] - start_color[k])) if k < 3 else start_color[k]
#                 for k in range(4)
#             )
#             rgba_string = f"rgba({interpolated_color[0]}, {interpolated_color[1]}, {interpolated_color[2]}, {interpolated_color[3]})"
#             gmt_panoply_256.append([round(pos, 3), rgba_string])
#             break

# gmt_panoply_256  # Display the first 10 color stops to verify
